## Below is the location of file on my local
/home/logic/Downloads/export00.csv

### checking sc version

In [22]:
sc.version

u'1.6.2'

## Loading File by sc

In [2]:
csv_data = sc.textFile("/home/logic/Downloads/export00.csv")

## checking first Row

In [7]:
csv_data.first()

u'DATE_VALEUR;HEURE_VALEUR;VALEUR;ID_EQUIPEMENT;LIBELLE_EQUIPEMENT;ID_GENIECIVIL;LIBELLE_GENIECIVIL;X_GC;Y_GC;ID_STATION;LIBELLE_STATION;X_STATION;Y_STATION;ID_RESEAU;LIBELLE_RESEAU;ID_PARAMETRE;LIBELLE_PARAMETRE;ID_UNITE;LIBELLE_UNITE'

## (;) semi collon separte eache row (conver string to list)

In [12]:
seprated_csv_data = csv_data.map(lambda x:x.split(";"))

#### Assigning first row into header_row variable

In [15]:
header_row = seprated_csv_data.first()
header_row

[u'DATE_VALEUR',
 u'HEURE_VALEUR',
 u'VALEUR',
 u'ID_EQUIPEMENT',
 u'LIBELLE_EQUIPEMENT',
 u'ID_GENIECIVIL',
 u'LIBELLE_GENIECIVIL',
 u'X_GC',
 u'Y_GC',
 u'ID_STATION',
 u'LIBELLE_STATION',
 u'X_STATION',
 u'Y_STATION',
 u'ID_RESEAU',
 u'LIBELLE_RESEAU',
 u'ID_PARAMETRE',
 u'LIBELLE_PARAMETRE',
 u'ID_UNITE',
 u'LIBELLE_UNITE']

#### now filter out first row of data

In [19]:
final_data_rdd = seprated_csv_data.filter(lambda row:row != header_row)
final_data_rdd.take(1)

[[u'20180103',
  u'10000',
  u'2441,3',
  u'1307374417',
  u'Pompe de rel\xe8vement',
  u'-183797836',
  u'Poste de refoulement',
  u'844610',
  u'6629692',
  u'null',
  u'/',
  u'null',
  u'null',
  u'60871189001',
  u'EPERVANS',
  u'NEP01',
  u'Temps de fonctionnement',
  u'59',
  u'heure ']]

## Convert this rdd to dataFrame

In [25]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [26]:
final_data = final_data_rdd.toDF(header_row)
final_data.show()

+-----------+------------+-------+-------------+--------------------+-------------+--------------------+------+-------+----------+---------------+---------+---------+-----------+--------------+------------+--------------------+--------+-------------+
|DATE_VALEUR|HEURE_VALEUR| VALEUR|ID_EQUIPEMENT|  LIBELLE_EQUIPEMENT|ID_GENIECIVIL|  LIBELLE_GENIECIVIL|  X_GC|   Y_GC|ID_STATION|LIBELLE_STATION|X_STATION|Y_STATION|  ID_RESEAU|LIBELLE_RESEAU|ID_PARAMETRE|   LIBELLE_PARAMETRE|ID_UNITE|LIBELLE_UNITE|
+-----------+------------+-------+-------------+--------------------+-------------+--------------------+------+-------+----------+---------------+---------+---------+-----------+--------------+------------+--------------------+--------+-------------+
|   20180103|       10000| 2441,3|   1307374417| Pompe de relèvement|   -183797836|Poste de refoulement|844610|6629692|      null|              /|     null|     null|60871189001|      EPERVANS|       NEP01|Temps de fonction...|      59|       heur

## Read these comments
All above process we have done in spark version 1.6 if you have spark > 2.0 then use below

final_data = spark.read.format("csv").option("header", "true").load("/home/logic/Downloads/export00.csv")

#### Count total records 

In [28]:
final_data.count()

433201

#### Count distinct DATE_VALEUR

In [29]:
final_data.select("DATE_VALEUR").distinct().count()

27

#### Count distinct DATE_VALEUR

In [30]:
final_data.select("LIBELLE_EQUIPEMENT").distinct().count()

14

#### Check some stats

In [35]:
final_data.describe().show(truncate=False)

+-------+
|summary|
+-------+
|count  |
|mean   |
|stddev |
|min    |
|max    |
+-------+



#### Count distinct LIBELLE_EQUIPEMENT in each DATE_VALEUR

In [39]:
from pyspark.sql import functions as F

In [40]:
final_data.groupby("DATE_VALEUR").agg(F.countDistinct("LIBELLE_EQUIPEMENT")).show()

+-----------+-------------------------+
|DATE_VALEUR|count(LIBELLE_EQUIPEMENT)|
+-----------+-------------------------+
|   20180101|                       14|
|   20180102|                       14|
|   20180103|                       14|
|   20180104|                       14|
|   20180105|                       14|
|   20180106|                       14|
|   20180107|                       14|
|   20180108|                       14|
|   20180109|                       14|
|   20180110|                       14|
|   20180111|                       14|
|   20180112|                       14|
|   20180113|                       14|
|   20180114|                       14|
|   20180115|                       14|
|   20180116|                       14|
|   20180117|                       14|
|   20180118|                       14|
|   20180119|                       14|
|   20180120|                       14|
+-----------+-------------------------+
only showing top 20 rows



Each LIBELLE_EQUIPEMENT has been used in each week